In [13]:
# package
import json  # For handling JSON data
from openai import OpenAI  # For interacting with OpenAI API
import os  # For interacting with the operating system, such as file paths
import re  # For regular expressions, useful for pattern matching in strings
import pandas as pd

In [117]:
# block 1 recording the final taxonomic key results
# Import knowledge graph
with open("D:/桌面/knowledge_graph.json", 'r') as file:
    knowledge_graph = json.load(file)

# Sample data for classification results
classification_results = {
    '1': '{\n    "Character": "Character5",\n    "States": {\n        "1": ["Coccygidium"],\n        "2": {\n            "Character": "Character13",\n            "States": {\n                "1": ["Cremnoptoides"],\n                "2": {\n                    "Character": "Character10",\n                    "States": {\n                        "1": ["Disophrys"],\n                        "2": ["Biroia"]\n                    }\n                }\n            }\n        }\n    }\n}',
    '2': '{\n    "Character": "Character 1",\n    "States": {\n        "1 and 2": ["Cremnops"],\n        "2": {\n            "Character": "Character 9",\n            "States": {\n                "1": ["Hypsostypos"],\n                "2": ["Troticus"]\n            }\n        }\n    }\n}',
    '4': '{\n    "Character": "Character2",\n    "States": {\n        "1": ["Earinus"],\n        "2": {\n            "Character": "Character3",\n            "States": {\n                "1": {\n                    "Character": "Character10",\n                    "States": {\n                        "1": {\n                            "Character": "Character13",\n                            "States": {\n                                "1": ["Amputostypos"],\n                                "2": ["Euagathis"]\n                            }\n                        },\n                        "2": ["Earinus"]\n                    }\n                },\n                "2": {\n                    "Character": "Character4",\n                    "States": {\n                        "1": ["Braunsia"],\n                        "2": {\n                            "Character": "Character9",\n                            "States": {\n                                "1": {\n                                    "Character": "Character14",\n                                    "States": {\n                                        "1": ["Lytopylus"],\n                                        "2": ["Agathis"]\n                                    }\n                                },\n                                "1 and 2": ["Therophilus"]\n                            }\n                        }\n                    }\n                },\n                "3": ["Bassus"]\n            }\n        },\n        "3": {\n            "Character": "Character8",\n            "States": {\n                "1": ["Aneurobracon"],\n                "3": ["Camptothlipsis"]\n            }\n        }\n    }\n}'
}

# Function to extract paths of characteristics and states recursively
def extract_paths(node, path=None):
    if path is None:
        path = {}

    if 'Character' in node and 'States' in node:
        current_character = node['Character'].replace(" ", "").strip()
        for state, value in node['States'].items():
            new_path = path.copy()
            new_path[current_character] = state
            if isinstance(value, dict):
                yield from extract_paths(value, new_path)
            else:
                for species in value:
                    yield species, new_path

# Process each classification result and extract paths
final_results = {}
for key, json_str in classification_results.items():
    classification_data = json.loads(json_str)
    species_paths = list(extract_paths(classification_data))

    formatted_results = {}
    for species, path in species_paths:
        formatted_results[species] = {"Characteristics": path}
    
    final_results[key] = formatted_results



# block 2: compare with the original knowledge graph, and recording the errors part
# Compare results and record differences
comparison_results = {}
errors = []
# Helper function to check if the state matches the correct state
def check_state_match(state, correct_state):
    if correct_state is None:
        return False
    if " and " in correct_state:
        correct_states = correct_state.split(" and ")
        return all(sub_state in correct_states for sub_state in state.split(" and "))
    return state == correct_state

# Add more logging for debugging
for key, results in final_results.items():
    for species, data in results.items():
        if species in knowledge_graph:
            mismatch = False
            incorrect_character_states = {}
            for character, state in data["Characteristics"].items():
                # Correct the character name format
                character = character.replace(" ", "").strip()
                correct_state = knowledge_graph[species]["Characteristics"].get(character)
                if correct_state is None:
                    print(f"Correct state not found for {species} ({character}): error_state = {state}")
                if correct_state is None or not check_state_match(state, correct_state):
                    mismatch = True
                    incorrect_character_states[character] = {"error_state": state, "correct_state": correct_state}
                    print(f"Character mismatch for {species} ({character}): error_state = {state}, correct_state = {correct_state}")
                else:
                    print(f"Character match for {species} ({character}): error_state = {state}, correct_state = {correct_state}")
            if mismatch:
                errors.append({
                    "species": species,
                    "key": key,
                    "error": "Mismatch",
                    "error_result": incorrect_character_states,
                    "correct_result": {character: knowledge_graph[species]["Characteristics"].get(character) for character in incorrect_character_states}
                })
        else:
            errors.append({
                "species": species,
                "key": key,
                "error": "Species not found in knowledge graph",
                "error_result": data["Characteristics"]
            })

# Output results
# print("Classification Results:")
# print(json.dumps(final_results, indent=4))

print("\nErrors:")
print(json.dumps(errors, indent=4))
print(errors)


Character match for Coccygidium (Character5): error_state = 1, correct_state = 1
Character match for Cremnoptoides (Character5): error_state = 2, correct_state = 2
Character match for Cremnoptoides (Character13): error_state = 1, correct_state = 1
Character match for Disophrys (Character5): error_state = 2, correct_state = 2
Character match for Disophrys (Character13): error_state = 2, correct_state = 2
Character match for Disophrys (Character10): error_state = 1, correct_state = 1
Character match for Biroia (Character5): error_state = 2, correct_state = 2
Character match for Biroia (Character13): error_state = 2, correct_state = 2
Character match for Biroia (Character10): error_state = 2, correct_state = 2
Character match for Cremnops (Character1): error_state = 1 and 2, correct_state = 1 and 2
Character match for Hypsostypos (Character1): error_state = 2, correct_state = 2
Character match for Hypsostypos (Character9): error_state = 1, correct_state = 1
Character match for Troticus (C

1. Initialization and data loading

In [118]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
with open("D:/桌面/knowledge_graph.json", 'r') as file:
    knowledge_graph = json.load(file)
    
classification_results = {'1': '{\n    "Character": "Character 5",\n    "States": {\n        "1": ["Coccygidium"],\n        "2": {\n            "Character": "Character 10",\n            "States": {\n                "1": ["Disophrys"],\n                "2": {\n                    "Character": "Character 13",\n                    "States": {\n                        "1": ["Cremnoptoides"],\n                        "2": ["Biroia"]\n                    }\n                }\n            }\n        }\n    }\n}', '2': '{\n    "Character": "Character 1",\n    "States": {\n        "2": {\n            "Character": "Character 9",\n            "States": {\n                "2": ["Troticus"],\n                "1": ["Hypsostypos"]\n            }\n        },\n        "1 and 2": ["Cremnops"]\n    }\n}', '4': '{\n    "Character": "Character2",\n    "States": {\n        "1 and 2": {\n            "Character": "Character3",\n            "States": {\n                "2": {\n                    "Character": "Character4",\n                    "States": {\n                        "2": {\n                            "Character": "Character6",\n                            "States": {\n                                "1": ["Agathis"],\n                                "2": ["Lytopylus"]\n                            }\n                        },\n                        "1": ["Braunsia"]\n                    }\n                },\n                "1": {\n                    "Character": "Character10",\n                    "States": {\n                        "1": ["Amputostypos"],\n                        "1 and 2": ["Euagathis"]\n                    }\n                }\n            }\n        },\n        "3": {\n            "Character": "Character8",\n            "States": {\n                "3": {\n                    "Character": "Character3",\n                    "States": {\n                        "2 and 3": ["Aneurobracon"],\n                        "2": ["Camptothlipsis"]\n                    }\n                }\n            }\n        },\n        "1": ["Earinus"],\n        "2": {\n            "Character": "Character3",\n            "States": {\n                "2": ["Therophilus"],\n                "3": ["Bassus"]\n            }\n        }\n    }\n}'}

In [119]:
import re
import json

# 初始响应
initial_response = {
    "Character": "Character1",
    "States": {
        "1": ["Biroia", "Coccygidium", "Disophrys", "Cremnoptoides"],
        "2": ["Troticus", "Hypsostypos", "Cremnops"],
        "3": ["Gyrochus"],
        "4": ["Agathis", "Lytopylus", "Braunsia", "Camptothlipsis", "Therophilus", "Bassus", "Earinus", "Amputostypos", "Euagathis", "Aneurobracon"]
    }
}

# 函数解析分类结果
def parse_classification_result(result_text):
    classification = {"Character": None, "States": {}}
    try:
        # 尝试匹配Character
        character_match = re.search(r'"Character": "([^"]+)"', result_text)
        if character_match:
            classification["Character"] = character_match.group(1)
        else:
            raise ValueError("Character not found in the result text.")

        # 尝试匹配各个State和对应的species
        state_sections = re.findall(r'"(\d+|[^"]+)":\s*\[(.*?)\]', result_text)
        if not state_sections:
            raise ValueError("No states found in the result text.")

        for state, species_block in state_sections:
            species_list = re.findall(r'"([^"]+)"', species_block)
            if not species_list:
                raise ValueError(f"No species found for state {state}.")
            classification["States"][state] = species_list

    except Exception as e:
        print(f"Error parsing classification result: {e}")
        raise e  # 或者 return classification

    return classification

# 将字典转换为字符串
initial_response_str = json.dumps(initial_response)

# 解析初始分类结果
parsed_initial_classification = parse_classification_result(initial_response_str)
print(parsed_initial_classification)

# 生成分类结果的组
def generate_groups_from_classification(classification_result):
    groups = []
    for state, species_list in classification_result["States"].items():
        groups.append((state, species_list))
    return groups

# 从解析的初始分类结果生成组
groups = generate_groups_from_classification(parsed_initial_classification)
print(type(groups))

{'Character': 'Character1', 'States': {'1': ['Biroia', 'Coccygidium', 'Disophrys', 'Cremnoptoides'], '2': ['Troticus', 'Hypsostypos', 'Cremnops'], '3': ['Gyrochus'], '4': ['Agathis', 'Lytopylus', 'Braunsia', 'Camptothlipsis', 'Therophilus', 'Bassus', 'Earinus', 'Amputostypos', 'Euagathis', 'Aneurobracon']}}
<class 'list'>


2. Extraction path
Define a recursive function extract_paths to extract the classification paths for each species.
Process each classification result, extract the paths and format them into dictionary form to be stored in final_results.

In [120]:
def extract_paths(node, path=None):
    if path is None:
        path = {}

    if 'Character' in node and 'States' in node:
        current_character = node['Character'].replace(" ", "").strip()
        for state, value in node['States'].items():
            new_path = path.copy()
            new_path[current_character] = state
            if isinstance(value, dict):
                yield from extract_paths(value, new_path)
            else:
                for species in value:
                    yield species, new_path

# 处理每个分类结果并提取路径
final_results = {}

for key, json_str in classification_results.items():
    classification_data = json.loads(json_str)
    species_paths = list(extract_paths(classification_data))

    formatted_results = {}
    for species, path in species_paths:
        formatted_results[species] = {"Characteristics": path}
    
    final_results[key] = formatted_results
    

3. Validation of classification results and recording of errors
Define the check_state_match function to check for state matches.
Define validate_results function to validate classification results and log errors.

In [121]:
def check_state_match(state, correct_state):
    if correct_state is None:
        return False
    if " and " in correct_state:
        correct_states = correct_state.split(" and ")
        return all(sub_state in correct_states for sub_state in state.split(" and "))
    return state == correct_state

# Validate classification results and log errors
def validate_results(final_results, knowledge_graph):
    errors = []
    for key, results in final_results.items():
        for species, data in results.items():
            if species in knowledge_graph:
                mismatch = False
                incorrect_character_states = {}
                for character, state in data["Characteristics"].items():
                    character = character.replace(" ", "").strip()
                    correct_state = knowledge_graph[species]["Characteristics"].get(character)
                    if correct_state is None or not check_state_match(state, correct_state):
                        mismatch = True
                        incorrect_character_states[character] = {"error_state": state, "correct_state": correct_state}
                if mismatch:
                    errors.append({
                        "species": species,
                        "key": key,
                        "error": "Mismatch",
                        "error_result": incorrect_character_states,
                        "correct_result": {character: knowledge_graph[species]["Characteristics"].get(character) for character in incorrect_character_states}
                    })
            else:
                errors.append({
                    "species": species,
                    "key": key,
                    "error": "Species not found in knowledge graph",
                    "error_result": data["Characteristics"]
                })
    return errors

4. Correction of classification results
Define the correct_classification function to call the API to correct the classification based on an error result.
Reclassify using the correct species feature state and update the classification result

In [122]:
import json

# 示例数据
errors = [
    {'species': 'Earinus', 'key': '4', 'error': 'Mismatch', 'error_result': {'Character2': {'error_state': '2', 'correct_state': '1'}, 'Character3': {'error_state': '1', 'correct_state': '2'}}, 'correct_result': {'Character2': '1', 'Character3': '2'}},
    {'species': 'Aneurobracon', 'key': '4', 'error': 'Mismatch', 'error_result': {'Character8': {'error_state': '1', 'correct_state': '3'}}, 'correct_result': {'Character8': '3'}}
]

groups = [
    {'Character': 'Character1', 'States': {'1': ['Biroia', 'Coccygidium', 'Disophrys', 'Cremnoptoides'], '2': ['Troticus', 'Hypsostypos', 'Cremnops'], '3': ['Gyrochus'], '4': ['Agathis', 'Lytopylus', 'Braunsia', 'Camptothlipsis', 'Therophilus', 'Bassus', 'Earinus', 'Amputostypos', 'Euagathis', 'Aneurobracon']}}
]

# 导入 knowledge_graph
with open("D:/桌面/knowledge_graph.json", 'r') as file:
    knowledge_graph = json.load(file)

# 处理每个错误
for error in errors:
    key = error['key']
    
    # 获取包含键 'key' 的物种列表
    species_list = []
    for group in groups:
        if 'States' in group and key in group['States']:
            species_list = group['States'][key]
            break

    if not species_list:
        print(f"Key {key} not found in any group['States']")
        continue

    print(f"Processing species list for state '{key}': {species_list}")
    
    # 构建 group_matrix_str
    group_matrix = {s: knowledge_graph[s] for s in species_list}
    group_matrix_str = json.dumps(group_matrix, ensure_ascii=False) 


Processing species list for state '4': ['Agathis', 'Lytopylus', 'Braunsia', 'Camptothlipsis', 'Therophilus', 'Bassus', 'Earinus', 'Amputostypos', 'Euagathis', 'Aneurobracon']
Processing species list for state '4': ['Agathis', 'Lytopylus', 'Braunsia', 'Camptothlipsis', 'Therophilus', 'Bassus', 'Earinus', 'Amputostypos', 'Euagathis', 'Aneurobracon']


In [123]:
# Function to clean and extract JSON string
def extract_json_string(json_string):
    # Find the positions of the start and end of the JSON object
    start = json_string.find('{')
    end = json_string.rfind('}') + 1
    
    # If both start and end positions are valid, extract and return the JSON string
    if start != -1 and end != -1:
        cleaned_string = json_string[start:end]
        return cleaned_string.strip()
    
    # If positions are not valid, return an empty string
    return ""


In [124]:
def correct_classification(errors, classification_results, knowledge_graph):
    for error in errors:
        key = error['key']
        
        species_list = []
        for group in groups:
            if 'States' in group and key in group['States']:
                species_list = group['States'][key]
                break
        if not species_list:
            print(f"Key {key} not found in any group['States']")
            continue
        print(f"Processing species list for state '{key}': {species_list}")
        
        group_matrix = {s: knowledge_graph[s] for s in species_list}
        group_matrix_str = json.dumps(group_matrix, ensure_ascii=False) 
        
        messages2 = [
            {"role": "system",
             "content": """
             You are a helpful taxonomist assistant.
             You are skilled at calculating the correct information gain to choose the character that best divides species into even groups based on their states.
             Based on the selected character, classify the species into different groups according to their states.
             For each group with more than two species, continue selecting characters to further classify this group until each group only has one species.
             After multiple classifications, determine the final classification levels and record each classifying character and its state.
             Finally, generate a taxonomic key.
             You are able to avoid the same error in your results based on the corrected results previously passed to you
             """},
            {"role": "system",
             "content": """
             Generate the nested taxonomic key based on the provided morphological matrix.
             The process involves selecting a character to classify the species into groups. Repeat this classification within each subgroup until each group contains only one species.
             Information gain measures how much the uncertainty in the dataset is reduced after using a character for classification. It helps in selecting characters that minimize the entropy of the subset after classification, leading to better classification results.
             Please select the classification character for these group's species based on the morphological matrix and information gain methods.
             In the morphological matrix, 'Missing' and 'Not applicable' are invalid states. If a character has invalid states for the group being classified, it should be ignored.
             States are represented by numbers. For example, '1 and 2' means multiple states should be treated as a single state type and this multi-state characterization should not be confused with the single states within it (the state of '3' and '2 and 3' is different state, when you choose the character to based on the state to distinguish need to careful handle). The initial character should have no more than three state types.
             You need to calculate the information gain for each character and choose the highest information gain result. The higher the information gain result, the greater the contribution of the feature to the classification.
             After selecting the initial classification character and categorizing the species based on its state, repeat the process within each subgroup. For each subgroup, select the character with the highest information gain to further classify the species. Continue this process recursively until each group contains only one species.
             Now I will show you the morphological matrix. Please provide the classification character and the categorization of species based on its state. Then, continue to classify each subgroup recursively, showing the chosen character and categorization for each subgroup. Please present the result in a structured format, with each step clearly labeled.
             Please don't show how you analyze and calculate, please show me the final result.
             """},
            {"role": "user", "content": f"""
            This is the result of the error you generated in the previous API call. 
            In this file I have provided you with the CORRECT result. 
            Please strictly adhere to the use of the correct species feature status message!{error}
            """},
            {"role": "assistant", "content": f"""
            I will strictly use the correct species feature state information for evaluation, 
            while I will avoid using these incorrect feature state information that appeared previously in the classification results
            """},
            {"role": "user", "content": f"Here is the group information need to be classify and include the morphological matrix {group_matrix_str}"}
        ]
        
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages2,
            stop=None,
            temperature=0,
            max_tokens=1000,
            n=1
        )
        corrected_result = response.choices[0].message.content
        
        messages_JSON = [
        {"role": "system",
         "content":
             """
             You are a helpful JSON format converter.
             You can express the nested structure as a JSON result based on the corresponding content.
             """},
        {"role": "system",
         "content":
             """
             Please format the classification result as follows:
             ```
             # Final taxonomic key result JSON format #
             {
                 "Character": "CharacterX",
                 "States": {
                     "1": ["speciesA"],
                     "2": {
                         "Character": "CharacterY",
                         "States": {
                             "1": ["speciesB"],
                             "2": ["speciesC"]
                         }
                     }
                 }
             }
             ```
             Ensure that the response follows this format exactly.
             """},
        {"role": "assistant",
         "content":
             """
             Understood. I'll convert the nested structure you gave me into JSON format and store it in # final result #.
             Please provide what you need to convert the format.
             """},
        {"role": "user", "content": f"Here are the taxonomic results for the nested schema representation {corrected_result}"}
        ]
    
        # Make the API call to format the response as JSON
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages_JSON,
            stop=None,
            temperature=0,
            max_tokens=1500,
            n=1
        )
        json_result = response.choices[0].message.content
        json_cleaned_result = extract_json_string(json_result)
        print(json_cleaned_result)
        classification_results[key] = json_cleaned_result
        return classification_results

5. Cycle checks and corrections
Ensure that classification results are correct through validation and correction cycles.
Validate classification results and log errors.
Call the API to correct the incorrect classification results.
Update the classification results and validate again, repeat until all errors are corrected.
Save the final result to a file.

In [125]:
# Cycle checks and corrections
errors = validate_results(final_results, knowledge_graph)
# Purpose: Enter a loop until all errors have been fixed.
# Function: Executes the code inside the loop when the errors list is not empty.
while errors:
    # Fix current categorization errors. (based on the API )
    classification_results = correct_classification(errors, classification_results, knowledge_graph)
    # Purpose: To reset the final_results dictionary to store the corrected categorization results.
    final_results = {}
    # Iterate over the corrected classification results and extract species classification paths.
    for key, json_str in classification_results.items():
        classification_data = json.loads(json_str)
        species_paths = list(extract_paths(classification_data))
        # Purpose: Format the extracted classification paths and store them in the formatted_results dictionary.
        formatted_results = {}
        for species, path in species_paths:
            formatted_results[species] = {"Characteristics": path}
        # Purpose: Add the formatted classification results to final_results.
        final_results[key] = formatted_results
    # Purpose: Re-validate the corrected classification results and log any remaining errors.
    # Function: Call the validate_results function, passing in the updated final_results and knowledge_graph and returning a new list of errors. If there are no errors, then errors is empty and the loop ends.
    errors = validate_results(final_results, knowledge_graph)

# Save the final classification results
with open('final_classification.json', 'w') as f:
    json.dump(final_results, f, indent=4)

print("Final classification results have been saved to 'final_classification.json'.")
print(json.dumps(final_results, indent=4))

Processing species list for state '4': ['Agathis', 'Lytopylus', 'Braunsia', 'Camptothlipsis', 'Therophilus', 'Bassus', 'Earinus', 'Amputostypos', 'Euagathis', 'Aneurobracon']
{
    "Character": "Character2",
    "States": {
        "1": ["Earinus"],
        "2": {
            "Character": "Character3",
            "States": {
                "1": ["Amputostypos", "Euagathis"],
                "2": ["Agathis", "Braunsia"]
            }
        },
        "3": {
            "Character": "Character3",
            "States": {
                "2": ["Lytopylus", "Therophilus"],
                "3": ["Bassus"]
            }
        },
        "4": {
            "Character": "Character16",
            "States": {
                "1": ["Aneurobracon"],
                "2": ["Camptothlipsis"]
            }
        }
    }
}
Processing species list for state '4': ['Agathis', 'Lytopylus', 'Braunsia', 'Camptothlipsis', 'Therophilus', 'Bassus', 'Earinus', 'Amputostypos', 'Euagathis', 'Aneurobracon']

In [126]:
print(json.dumps(classification_results))

{"1": "{\n    \"Character\": \"Character 5\",\n    \"States\": {\n        \"1\": [\"Coccygidium\"],\n        \"2\": {\n            \"Character\": \"Character 10\",\n            \"States\": {\n                \"1\": [\"Disophrys\"],\n                \"2\": {\n                    \"Character\": \"Character 13\",\n                    \"States\": {\n                        \"1\": [\"Cremnoptoides\"],\n                        \"2\": [\"Biroia\"]\n                    }\n                }\n            }\n        }\n    }\n}", "2": "{\n    \"Character\": \"Character 1\",\n    \"States\": {\n        \"2\": {\n            \"Character\": \"Character 9\",\n            \"States\": {\n                \"2\": [\"Troticus\"],\n                \"1\": [\"Hypsostypos\"]\n            }\n        },\n        \"1 and 2\": [\"Cremnops\"]\n    }\n}", "4": "{\n    \"Character\": \"Character2\",\n    \"States\": {\n        \"1 and 2\": {\n            \"Character\": \"Character3\",\n            \"States\": {\n    

In [127]:
# Example initial result
initial_result = parsed_initial_classification

# Parse the API response JSON strings
parsed_classification_results = {key: json.loads(value) for key, value in classification_results.items()}

# Function to combine the initial and secondary classification results
def combine_results(initial, secondary, state_key):
    if not secondary:
        return

    initial_states = initial["States"].get(state_key)
    if initial_states is None:
        initial["States"][state_key] = secondary
        return

    if isinstance(initial_states, list):
        if isinstance(secondary, list):
            initial["States"][state_key] = list(set(initial_states + secondary))  # Merge two lists and remove duplicates
        else:
            initial["States"][state_key] = secondary
    elif isinstance(initial_states, dict):
        if isinstance(secondary, dict):
            for key, value in secondary["States"].items():
                if key not in initial_states:
                    initial_states[key] = value
                else:
                    combine_results(initial_states, value, key)
        else:
            raise ValueError(f"Conflicting types for key {state_key}: {type(initial_states)} vs {type(secondary)}")
    else:
        raise ValueError(f"Unexpected type for initial states: {type(initial_states)}")

# Dynamically combine all secondary classification results
for state_key, secondary in parsed_classification_results.items():
    combine_results(initial_result, secondary, state_key)

# Function to display the final classification result
def display_classification(result, indent=0):
    indent_space = " " * indent
    character = result.get("Character")
    states = result.get("States")

    classification = {}
    if character and states:
        classification["Character"] = character
        classification["States"] = {}
        print(f"{indent_space}1. **{character}:**")
        for state, species in states.items():
            if isinstance(species, list):
                print(f"{indent_space}   - State \"{state}\": {', '.join(species)}")
                classification["States"][state] = species
            elif isinstance(species, dict):
                print(f"{indent_space}   - State \"{state}\":")
                classification["States"][state] = display_classification(species, indent + 4)
    return classification

# Display the final classification result
final_result = display_classification(initial_result)
print("\nFinal Result JSON:")
# print(json.dumps(final_result, indent=2))


1. **Character1:**
   - State "1":
    1. **Character 5:**
       - State "1": Coccygidium
       - State "2":
        1. **Character 10:**
           - State "1": Disophrys
           - State "2":
            1. **Character 13:**
               - State "1": Cremnoptoides
               - State "2": Biroia
   - State "2":
    1. **Character 1:**
       - State "2":
        1. **Character 9:**
           - State "2": Troticus
           - State "1": Hypsostypos
       - State "1 and 2": Cremnops
   - State "3": Gyrochus
   - State "4":
    1. **Character2:**
       - State "1 and 2": Amputostypos
       - State "2":
        1. **Character3:**
           - State "2":
            1. **Character4:**
               - State "1 and 2": Lytopylus
               - State "2": Therophilus
           - State "3": Bassus
       - State "3":
        1. **Character8:**
           - State "3": Camptothlipsis, Aneurobracon
       - State "1": Earinus

Final Result JSON:
